In [ ]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize
from PIL import Image

In [ ]:
#Pytorch device check for ROCM for AMD GPU
#https://github.com/pytorch/pytorch/issues/10670
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using {device} device")

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        label = 0 if img_name.startswith('cat') else 1  # Assign label 0 for cats and 1 for dogs


        return image, label

In [ ]:
train_path_dir = 'data/train'
test_path_dir = 'data/test1'
batch_size=64
num_workers = 4
image_size = (256, 256)  # Specify the desired image size
learning_rate = 1e-3
epochs = 10

transform = transforms.Compose([
    Resize(image_size),
    ToTensor()
])

In [ ]:
# Import data
train_data = CatDogDataset(train_path_dir, transform=transform)
test_data = CatDogDataset(test_path_dir, transform=transform)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:

## Get the first few images from the dataset
#num_images_to_display = 5
#sample_images = [train_data[i][0] for i in range(num_images_to_display)]
#
## Convert the tensor images back to PIL images
#pil_images = [transforms.ToPILImage()(image) for image in sample_images]
#
## Display the resized images
#for image in pil_images:
#    image.show()

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(256 * 256 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(model, train_dataloader, loss_fn, optimizer, device):
    model.train()
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        num_images_to_display = 5
        sample_images = [train_data[i][0] for i in range(num_images_to_display)]
        
        # Convert the tensor images back to PIL images
        pil_images = [transforms.ToPILImage()(image) for image in sample_images]
        
        # Display the resized images
        for image in pil_images:
            image.show()
        for label in range(0,4):
            print(labels[label])

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    return model

In [ ]:
def test_model(model, test_dataloader, device):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            print(labels, predicted, correct, total)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [ ]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    model = train_model(model, train_dataloader, loss_fn, optimizer, device)
    accuracy = test_model(model, test_dataloader, device)
    print(f"Epoch [{epoch+1}/{epochs}], Test Accuracy: {accuracy:.2f}%")

print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", map_location=device))